In [1]:
pip install boto3

  Obtaining dependency information for boto3 from https://files.pythonhosted.org/packages/01/c8/fcf1ec25b0320af58be3a3d8c0f8ed06513133c65b318895e7eb39ef14ab/boto3-1.35.66-py3-none-any.whl.metadata
  Obtaining dependency information for botocore<1.36.0,>=1.35.66 from https://files.pythonhosted.org/packages/92/f0/372c2474cd198485ac427ccdd54796b05c2e730bdff0523c26012fe40ad7/botocore-1.35.66-py3-none-any.whl.metadata
  Obtaining dependency information for s3transfer<0.11.0,>=0.10.0 from https://files.pythonhosted.org/packages/66/05/7957af15543b8c9799209506df4660cba7afc4cf94bfb60513827e96bed6/s3transfer-0.10.4-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 36.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.2/83.2 kB 11.7 MB/s eta 0:00:00
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.76
    Uninstalling botocore-1.29

In [1]:
pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.


In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import boto3
import logging

In [12]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from urllib.parse import urljoin
import string
import logging
import re
from datetime import datetime
import random
import os
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

class BasketballReferenceScraper:
    def __init__(self):
        self.base_url = 'https://www.basketball-reference.com/players'
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.session = self._setup_session()
        
    def _setup_session(self):
        """Set up session with retry strategy"""
        session = requests.Session()
        retry_strategy = Retry(
            total=5,  # number of retries
            backoff_factor=2,  # wait 1, 2, 4, 8, 16... seconds between retries
            status_forcelist=[429, 500, 502, 503, 504],
        )
        adapter = HTTPAdapter(max_retries=retry_strategy)
        session.mount("https://", adapter)
        session.mount("http://", adapter)
        return session

    def get_soup(self, url):
        """Make HTTP request and return BeautifulSoup object with rate limiting"""
        try:
            # Random delay between 2-4 seconds
            time.sleep(random.uniform(2, 4))
            
            response = self.session.get(url, headers=self.headers)
            
            if response.status_code == 429:
                wait_time = int(response.headers.get('Retry-After', 60))
                logger.info(f"Rate limited. Waiting {wait_time} seconds...")
                time.sleep(wait_time)
                response = self.session.get(url, headers=self.headers)
                
            response.raise_for_status()
            return BeautifulSoup(response.text, 'html.parser')
        except requests.RequestException as e:
            logger.error(f"Failed to fetch {url} - Error: {str(e)}")
            if "429" in str(e):
                logger.info("Rate limit hit. Waiting 60 seconds...")
                time.sleep(60)
            return None

    def download_csv(self, df, filename):
        """Download DataFrame as CSV"""
        try:
            df.to_csv(filename, index=False)
            logger.info(f"Successfully downloaded data to {filename}")
            
            # Basic statistics about the data
            logger.info(f"Total players: {len(df)}")
            logger.info(f"Columns: {', '.join(df.columns)}")
            logger.info(f"File size: {os.path.getsize(filename) / 1024:.2f} KB")
            
            return True
        except Exception as e:
            logger.error(f"Error saving CSV: {str(e)}")
            return False

    def get_alphabet_links(self):
        """Generate alphabet index page links"""
        return [f"{self.base_url}/{letter}" for letter in string.ascii_lowercase]

    def get_player_links(self, alphabet_url):
        """Get all player links from an alphabet index page"""
        soup = self.get_soup(alphabet_url)
        if not soup:
            return []
        
        player_links = []
        players_table = soup.find('table', id='players')
        if players_table:
            for link in players_table.select('th[data-stat="player"] a'):
                href = link.get('href', '')
                if href:
                    player_url = urljoin(self.base_url, href)
                    player_links.append(player_url)
        
        return player_links

    def safe_get_text(self, element, default=None):
        """Safely extract text from an element"""
        if element is not None:
            return element.text.strip()
        return default

    def extract_player_stats(self, player_url):
        """Extract statistics for a single player"""
        soup = self.get_soup(player_url)
        if not soup:
            logger.error(f"Could not get soup for {player_url}")
            return None

        try:
            # Find player info section
            info_div = soup.find('div', {'id': 'meta'})
            if not info_div:
                logger.error(f"Could not find meta div for {player_url}")
                return None

            # Get player name from h1 span tag
            name_element = soup.find('h1').find('span')
            player_name = self.safe_get_text(name_element)
            if not player_name:
                logger.error(f"Could not find player name for {player_url}")
                return None
            
            bio_info = {
                'position': None,
                'shoots': None,
                'height': None,
                'weight_lb': None,
                'weight_kg': None,
                'height_cm': None,
                'birthdate': None,
                'birthplace': None,
                'college': None,
                'high_school': None,
                'draft': None,
                'nba_debut': None
            }

            # Extract information from paragraphs
            for p in info_div.find_all('p'):
                text = self.safe_get_text(p, '')
                
                if 'Position:' in text:
                    parts = text.split('•')
                    for part in parts:
                        part = part.strip()
                        if 'Position:' in part:
                            bio_info['position'] = part.replace('Position:', '').strip()
                        if 'Shoots:' in part:
                            bio_info['shoots'] = part.replace('Shoots:', '').strip()
                
                # Height and Weight
                if any(x in text for x in ['cm', 'kg', 'lb']):
                    match = re.search(r'(\d+-\d+), (\d+)lb \((\d+)cm, (\d+)kg\)', text)
                    if match:
                        bio_info['height'] = match.group(1)
                        bio_info['weight_lb'] = match.group(2)
                        bio_info['height_cm'] = match.group(3)
                        bio_info['weight_kg'] = match.group(4)

                if text.startswith('Born:'):
                    parts = text.split('in')
                    if len(parts) > 1:
                        bio_info['birthplace'] = parts[1].strip()
                        date_match = re.search(r'Born: (.*?) in', text)
                        if date_match:
                            try:
                                birth_date = datetime.strptime(date_match.group(1).strip(), '%B %d, %Y')
                                bio_info['birthdate'] = birth_date.strftime('%Y-%m-%d')
                            except ValueError:
                                pass

                if text.startswith('College:'):
                    bio_info['college'] = text.replace('College:', '').strip()
                
                if text.startswith('High School:'):
                    bio_info['high_school'] = text.replace('High School:', '').strip()
                
                if text.startswith('Draft:'):
                    bio_info['draft'] = text.replace('Draft:', '').strip()
                
                if text.startswith('NBA Debut:'):
                    bio_info['nba_debut'] = text.replace('NBA Debut:', '').strip()

            # Find career stats - Updated to match the actual HTML structure
            career_stats = {}
            stats_div = soup.find('div', {'class': 'stats_pullout'})
            if stats_div:
                # Get all stat sections (p1 and p2)
                stat_sections = stats_div.find_all('div', class_=['p1', 'p2'])
                for section in stat_sections:
                    # Process each stat div within the section
                    stat_divs = section.find_all('div')
                    for i in range(len(stat_divs)):
                        # Find stat label
                        label_span = stat_divs[i].find('span', class_='poptip')
                        if label_span:
                            label = label_span.find('strong')
                            if label:
                                stat_key = self.safe_get_text(label)
                                # Get the corresponding value from the next <p> tag
                                value_p = stat_divs[i].find_all('p')
                                if len(value_p) >= 2:  # Make sure we have at least 2 <p> tags
                                    stat_value = self.safe_get_text(value_p[1])
                                    if stat_key and stat_value:
                                        career_stats[stat_key] = stat_value

            # Combine all information
            player_info = {
                'player_name': player_name,
                'url': player_url,
                **bio_info,
                'games': career_stats.get('G', None),
                'points': career_stats.get('PTS', None),
                'rebounds': career_stats.get('TRB', None),
                'assists': career_stats.get('AST', None),
                'fg_percentage': career_stats.get('FG%', None),
                'fg3_percentage': career_stats.get('3P%', None),  # Updated key to match website
                'ft_percentage': career_stats.get('FT%', None),
                'efg_percentage': career_stats.get('eFG%', None)
            }
            
            return player_info

        except Exception as e:
            logger.error(f"Error processing {player_url} - Error: {str(e)}")
            return None

    def scrape_all_players(self, output_file='basketball_reference_stats.csv', start_letter=None, end_letter=None):
        """Main function to scrape all players and save to CSV"""
        all_stats = []
        alphabet_links = self.get_alphabet_links()
        
        if start_letter:
            start_idx = string.ascii_lowercase.index(start_letter.lower())
            alphabet_links = alphabet_links[start_idx:]
        if end_letter:
            end_idx = string.ascii_lowercase.index(end_letter.lower()) + 1
            alphabet_links = alphabet_links[:end_idx]

        logger.info(f"Starting scrape for letters: {[link.split('/')[-1] for link in alphabet_links]}")

        for alphabet_url in alphabet_links:
            letter = alphabet_url.split('/')[-1]
            logger.info(f"Processing players for letter: {letter}")
            
            player_links = self.get_player_links(alphabet_url)
            logger.info(f"Found {len(player_links)} players for letter {letter}")

            for player_url in player_links:
                stats = self.extract_player_stats(player_url)
                if stats:
                    all_stats.append(stats)
                    logger.info(f"Successfully scraped stats for {stats['player_name']}")
                else:
                    logger.warning(f"Failed to extract stats for {player_url}")

            # Save progress after each letter
            if all_stats:
                df = pd.DataFrame(all_stats)
                temp_file = f'basketball_reference_stats_{letter}.csv'
                self.download_csv(df, temp_file)
                logger.info(f"Saved progress for letter {letter}")

        # Save final complete dataset
        if all_stats:
            df = pd.DataFrame(all_stats)
            success = self.download_csv(df, output_file)
            if success:
                logger.info(f"Successfully saved {len(all_stats)} player stats to {output_file}")
                return success
            else:
                logger.error("Failed to save final dataset")
        else:
            logger.warning("No player stats were collected")

def main():
    scraper = BasketballReferenceScraper()
    
    # Example: scrape only players with last names starting with A
    scraper.scrape_all_players(start_letter='a', end_letter='z')
    
    # Or scrape all players:
    # scraper.scrape_all_players()

if __name__ == "__main__":
    main()

2024-12-27 19:53:01,619 - INFO - Starting scrape for letters: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
2024-12-27 19:53:01,620 - INFO - Processing players for letter: a
2024-12-27 19:53:04,542 - INFO - Found 178 players for letter a
2024-12-27 19:53:08,914 - INFO - Successfully scraped stats for Alaa Abdelnaby
2024-12-27 19:53:11,188 - INFO - Successfully scraped stats for Zaid Abdul-Aziz
2024-12-27 19:53:15,013 - INFO - Successfully scraped stats for Kareem Abdul-Jabbar
2024-12-27 19:53:17,275 - INFO - Successfully scraped stats for Mahmoud Abdul-Rauf
2024-12-27 19:53:21,472 - INFO - Successfully scraped stats for Tariq Abdul-Wahad
2024-12-27 19:53:24,712 - INFO - Successfully scraped stats for Shareef Abdur-Rahim
2024-12-27 19:53:28,164 - INFO - Successfully scraped stats for Tom Abernethy
2024-12-27 19:53:32,135 - INFO - Successfully scraped stats for Forest Able
2024-12-27 19:53:35,577 - INFO 

2024-12-27 19:58:24,569 - INFO - Successfully scraped stats for Jerome Anderson
2024-12-27 19:58:27,879 - INFO - Successfully scraped stats for Justin Anderson
2024-12-27 19:58:30,601 - INFO - Successfully scraped stats for Kenny Anderson
2024-12-27 19:58:34,392 - INFO - Successfully scraped stats for Kim Anderson
2024-12-27 19:58:38,815 - INFO - Successfully scraped stats for Kyle Anderson
2024-12-27 19:58:41,802 - INFO - Successfully scraped stats for Michael Anderson
2024-12-27 19:58:44,454 - INFO - Successfully scraped stats for Nick Anderson
2024-12-27 19:58:48,288 - INFO - Successfully scraped stats for Richard Anderson
2024-12-27 19:58:52,000 - INFO - Successfully scraped stats for Ron Anderson
2024-12-27 19:58:55,609 - INFO - Successfully scraped stats for Ryan Anderson
2024-12-27 19:58:58,308 - INFO - Successfully scraped stats for Shandon Anderson
2024-12-27 19:59:01,758 - INFO - Successfully scraped stats for Willie Anderson
2024-12-27 19:59:05,743 - INFO - Successfully scra

2024-12-27 20:03:39,245 - INFO - Successfully scraped stats for Maurice Baker
2024-12-27 20:03:43,338 - INFO - Successfully scraped stats for Norm Baker
2024-12-27 20:03:47,395 - INFO - Successfully scraped stats for Ron Baker
2024-12-27 20:03:50,194 - INFO - Successfully scraped stats for Vin Baker
2024-12-27 20:03:53,469 - INFO - Successfully scraped stats for Patrick Baldwin Jr.
2024-12-27 20:03:56,319 - INFO - Successfully scraped stats for Wade Baldwin
2024-12-27 20:03:58,831 - INFO - Successfully scraped stats for Renaldo Balkman
2024-12-27 20:04:01,979 - INFO - Successfully scraped stats for Cedric Ball
2024-12-27 20:04:05,310 - INFO - Successfully scraped stats for LaMelo Ball
2024-12-27 20:04:09,133 - INFO - Successfully scraped stats for Lonzo Ball
2024-12-27 20:04:13,109 - INFO - Successfully scraped stats for Greg Ballard
2024-12-27 20:04:16,356 - INFO - Successfully scraped stats for Herschel Baltimore
2024-12-27 20:04:19,151 - INFO - Successfully scraped stats for Mo Bamb

2024-12-27 20:09:25,653 - INFO - Successfully scraped stats for Ernie Beck
2024-12-27 20:09:29,574 - INFO - Successfully scraped stats for Art Becker
2024-12-27 20:09:33,074 - INFO - Successfully scraped stats for Moe Becker
2024-12-27 20:09:36,697 - INFO - Successfully scraped stats for Bob Bedell
2024-12-27 20:09:40,858 - INFO - Successfully scraped stats for William Bedford
2024-12-27 20:09:43,709 - INFO - Successfully scraped stats for Reece Beekman
2024-12-27 20:09:46,701 - INFO - Successfully scraped stats for Hank Beenders
2024-12-27 20:09:49,204 - INFO - Successfully scraped stats for Ron Behagen
2024-12-27 20:09:53,048 - INFO - Successfully scraped stats for Elmer Behnke
2024-12-27 20:09:55,653 - INFO - Successfully scraped stats for Marco Belinelli
2024-12-27 20:09:57,983 - INFO - Successfully scraped stats for Charlie Bell
2024-12-27 20:10:00,928 - INFO - Successfully scraped stats for Dennis Bell
2024-12-27 20:10:03,557 - INFO - Successfully scraped stats for Jordan Bell
20

2024-12-27 20:15:05,376 - INFO - Successfully scraped stats for Buddy Boeheim
2024-12-27 20:15:09,257 - INFO - Successfully scraped stats for Tom Boerwinkle
2024-12-27 20:15:13,070 - INFO - Successfully scraped stats for Keith Bogans
2024-12-27 20:15:17,857 - INFO - Successfully scraped stats for Bogdan BogdanoviÄ
2024-12-27 20:15:21,882 - INFO - Successfully scraped stats for Bojan BogdanoviÄ
2024-12-27 20:15:25,232 - INFO - Successfully scraped stats for Muggsy Bogues
2024-12-27 20:15:29,666 - INFO - Successfully scraped stats for Andrew Bogut
2024-12-27 20:15:33,635 - INFO - Successfully scraped stats for Etdrick Bohannon
2024-12-27 20:15:37,838 - INFO - Successfully scraped stats for Bol Bol
2024-12-27 20:15:41,606 - INFO - Successfully scraped stats for Manute Bol
2024-12-27 20:15:45,403 - INFO - Successfully scraped stats for Jonah Bolden
2024-12-27 20:15:48,404 - INFO - Successfully scraped stats for Marques Bolden
2024-12-27 20:15:50,948 - INFO - Successfully scraped stats fo

...

2024-12-28 00:23:57,421 - INFO - Successfully scraped stats for Bob Warren
2024-12-28 00:24:00,557 - INFO - Successfully scraped stats for John Warren
2024-12-28 00:24:04,663 - INFO - Successfully scraped stats for T.J. Warren
2024-12-28 00:24:08,550 - INFO - Successfully scraped stats for Willie Warren
2024-12-28 00:24:10,976 - INFO - Successfully scraped stats for Bryan Warrick
2024-12-28 00:24:13,364 - INFO - Successfully scraped stats for Hakim Warrick
2024-12-28 00:24:15,677 - INFO - Successfully scraped stats for Chris Washburn
2024-12-28 00:24:19,789 - INFO - Successfully scraped stats for Julian Washburn
2024-12-28 00:24:23,926 - INFO - Successfully scraped stats for Bobby Washington
2024-12-28 00:24:26,494 - INFO - Successfully scraped stats for Darius Washington
2024-12-28 00:24:30,628 - INFO - Successfully scraped stats for Don Washington
2024-12-28 00:24:34,198 - INFO - Successfully scraped stats for Duane Washington
2024-12-28 00:24:38,181 - INFO - Successfully scraped sta

2024-12-28 00:29:47,876 - INFO - Successfully scraped stats for Randy White
2024-12-28 00:29:50,587 - INFO - Successfully scraped stats for Rodney White
2024-12-28 00:29:54,302 - INFO - Successfully scraped stats for Rory White
2024-12-28 00:29:58,122 - INFO - Successfully scraped stats for Royce White
2024-12-28 00:30:01,696 - INFO - Successfully scraped stats for Rudy White
2024-12-28 00:30:04,686 - INFO - Successfully scraped stats for Tony White
2024-12-28 00:30:07,966 - INFO - Successfully scraped stats for Willie White
2024-12-28 00:30:10,897 - INFO - Successfully scraped stats for Isaiah Whitehead
2024-12-28 00:30:14,068 - INFO - Successfully scraped stats for Jerome Whitehead
2024-12-28 00:30:17,320 - INFO - Successfully scraped stats for Donald Whiteside
2024-12-28 00:30:19,701 - INFO - Successfully scraped stats for Hassan Whiteside
2024-12-28 00:30:23,571 - INFO - Successfully scraped stats for Dwayne Whitfield
2024-12-28 00:30:27,213 - INFO - Successfully scraped stats for 

2024-12-28 00:35:26,961 - INFO - Successfully scraped stats for Micheal Williams
2024-12-28 00:35:29,627 - INFO - Successfully scraped stats for Mike Williams
2024-12-28 00:35:33,152 - INFO - Successfully scraped stats for Milt Williams
2024-12-28 00:35:36,454 - INFO - Successfully scraped stats for Mo Williams
2024-12-28 00:35:39,307 - INFO - Successfully scraped stats for Monty Williams
2024-12-28 00:35:41,983 - INFO - Successfully scraped stats for Nate Williams
2024-12-28 00:35:44,836 - INFO - Successfully scraped stats for Patrick Williams
2024-12-28 00:35:47,308 - INFO - Successfully scraped stats for Pete Williams
2024-12-28 00:35:51,234 - INFO - Successfully scraped stats for Ray Williams
2024-12-28 00:35:55,368 - INFO - Successfully scraped stats for Reggie Williams
2024-12-28 00:35:58,146 - INFO - Successfully scraped stats for Reggie Williams
2024-12-28 00:36:00,889 - INFO - Successfully scraped stats for Rickey Williams
2024-12-28 00:36:05,176 - INFO - Successfully scraped 

2024-12-28 00:41:10,457 - INFO - Successfully scraped stats for Sam Worthen
2024-12-28 00:41:14,054 - INFO - Successfully scraped stats for James Worthy
2024-12-28 00:41:17,355 - INFO - Successfully scraped stats for Antoine Wright
2024-12-28 00:41:20,360 - INFO - Successfully scraped stats for Bracey Wright
2024-12-28 00:41:23,774 - INFO - Successfully scraped stats for Brad Wright
2024-12-28 00:41:26,244 - INFO - Successfully scraped stats for Brandan Wright
2024-12-28 00:41:29,200 - INFO - Successfully scraped stats for Chris Wright
2024-12-28 00:41:31,868 - INFO - Successfully scraped stats for Chris Wright
2024-12-28 00:41:34,491 - INFO - Successfully scraped stats for Delon Wright
2024-12-28 00:41:37,177 - INFO - Successfully scraped stats for Dorell Wright
2024-12-28 00:41:39,590 - INFO - Successfully scraped stats for Howard Wright
2024-12-28 00:41:42,886 - INFO - Successfully scraped stats for Howie Wright
2024-12-28 00:41:46,587 - INFO - Successfully scraped stats for Joby Wr

2024-12-28 00:44:52,928 - INFO - Successfully saved 5284 player stats to basketball_reference_stats.csv
